# Set the working directory to the src folder on your machine

In [1]:
import os
os.chdir(r"C:\Users\osmou\Documents\GitHub\wingspan-bird-brain\src")
print(os.getcwd())

C:\Users\osmou\Documents\GitHub\wingspan-bird-brain\src


In [2]:
# Extension to autoreload code from other files when running code here
# Don't need to restart kernel in order to reflect changes in other files
%load_ext autoreload
%autoreload 2

In [43]:
from env import WingspanEnvironment
from Gameplay_Constants import BIRDFEEDER_FACES, FOOD_TYPES, MAX_NUMBER_DIE_IN_BIRDFEEDER
from Birdfeeder import Birdfeeder
import gymnasium
from gymnasium.spaces import Discrete, MultiDiscrete, Box, Text, MultiBinary
from gymnasium.spaces.utils import flatten_space
import numpy as np
from pettingzoo.test import api_test, parallel_api_test

In [67]:
env = WingspanEnvironment(render_mode="human", log=False)
_ = env.reset(seed=18)

In [75]:
# # Make sure action spaces, action masks, and action descriptions have the same keys
# for a in env.agents:
#     aspace_keys = list(env.action_space(a).keys())
#     amask_keys = list(env.observations[a]["action_mask"].keys())
#     adesc_keys = list(env.action_descriptions.keys())
#     assert np.all(aspace_keys == amask_keys), f"Action space and action mask for agent {a} do not have matching keys."
#     assert np.all(aspace_keys == adesc_keys), f"Action space for agent '{a}' and action descriptions do not have matching keys."

# # Make sure action spaces, action masks, and action descriptions are all the same shape
# for a in env.agents:
#     aspace = env.action_space(a)
#     amask = env.observations[a]["action_mask"]
#     adesc = env.action_descriptions
#     assert aspace.n == len(amask), f"Action space and action mask for agent {a} are not the same length"
#     assert aspace.n == len(adesc), f"Action space for agent {a} and action descriptions are not the same length"

In [45]:
print(env.game.birdfeeder)
print(env.game.players[env.agent_selection].food)
print(env.observations[env.agent_selection])

Birdfeeder: ['Fish', 'Invertebrate', 'Invertebrate+Seed', 'Fish', 'Fish']
{'Fish': 1, 'Rodent': 1, 'Fruit': 1, 'Invertebrate': 1, 'Seed': 1}
{'observation': {'Birdfeeder': [0, 3, 5, 0, 0]}, 'action_mask': {'Gain Food': array([1, 0, 0, 1, 0, 1, 1], dtype=int8), 'Reroll Birdfeeder': array([0], dtype=int8)}}


In [46]:
actions = {a: env.action_spaces[a].sample(env.observations[a]["action_mask"]) for a in env.agents}
env.step(actions)

{'bird-brain': OrderedDict({'Gain Food': 5, 'Reroll Birdfeeder': array([0], dtype=int8)})}
Taking Invertebrate+Seed and gaining Invertebrate.


In [47]:
print(env.game.birdfeeder)
print(env.game.players[env.agent_selection].food)
print(env.observe(env.agent_selection))

Birdfeeder: ['Fish', 'Invertebrate', 'Fish', 'Fish']
{'Fish': 1, 'Rodent': 1, 'Fruit': 1, 'Invertebrate': 2, 'Seed': 1}
{'observation': {'Birdfeeder': [0, 3, 0, 0, 6]}, 'action_mask': {'Gain Food': array([1, 0, 0, 1, 0, 0, 0], dtype=int8), 'Reroll Birdfeeder': array([0], dtype=int8)}}


In [ ]:
env.action_spaces[env.agent_selection].sample(env.observations[env.agent_selection]["action_mask"])

In [52]:
action_descriptions = {
    "Gain Food": [
        "Take Fish", "Take Rodent", "Take Fruit", "Take Invertebrate",
        "Take Seed", "Take Invert+Seed, gain Invertebrate", 
        "Take Invert+Seed, gain Seed",
    ],
    "Reroll Birdfeeder": ["Reroll Birdfeeder",],
}
flat_action_descriptions = np.concatenate(list(action_descriptions.values()))
print(flat_action_descriptions)

flat_action_space = flatten_space(env.action_space(env.agent_selection))
flat_action_space = Discrete(flat_action_space.shape[0])
print("\n", flat_action_space)

flat_action_mask = np.concatenate(list(env.observations[env.agent_selection]["action_mask"].values()))
print("\n", flat_action_mask.shape, flat_action_mask)

sampled_action = flat_action_space.sample(flat_action_mask)
print("\n", sampled_action, flat_action_descriptions[sampled_action])

['Take Fish' 'Take Rodent' 'Take Fruit' 'Take Invertebrate' 'Take Seed'
 'Take Invert+Seed, gain Invertebrate' 'Take Invert+Seed, gain Seed'
 'Reroll Birdfeeder']

 Discrete(8)

 (8,) [1 0 0 1 0 0 0 0]

 3 Take Invertebrate
